In [ ]:
from scipy.stats import t
import numpy.matlib as mb
import numpy.linalg as lg
import ipywidgets as widgets
import math as mt
import numpy as np
import pandas as pd
import re

In [ ]:
def erroMedicao(VVC, xi):
  n = np.size(xi,0) # n - Tamanho das amostas
  mVVC = mb.repmat(VVC,n,1) # Replica o vetor coluna VVc em n linhas
  em = (xi - mVVC)
  emMaxAbs = np.max(abs(em),0).T
  return (emMaxAbs)

def erroSistematico(VVC, xi):
  vm = np.mean(xi,0) # Média aritmética de cada linha
  return ((vm - VVC).T) # Erro sistemático ou Tendência

def erroAleatorio(xi):
  n = np.std(xi,0,ddof=1)
  return (n.T)

def histerese(xiAvanco,xiRecuo):
  vmAvanco = np.mean(xiAvanco,0) # Média aritmética em avanço.
  vmRecuo = np.mean(xiRecuo,0) # Média aritmética em recuo | retorno.
  hist = abs(vmAvanco-vmRecuo)
  histMax = np.max(hist,0)
  return (np.asmatrix(hist).T)

def linearidade(VVC, xi):
  tam = np.size(VVC,1) # Quantidade de valores analisados
  VVC = VVC.T
  vm = np.mean(xi,0).T # Média aritmética de cada linha
  # Calcula a reta com menor erro em relação as medidas - MMQ
  mt = np.concatenate((vm, np.ones((tam,1))),1)
  x = lg.inv(mt.T @ mt)
  y = mt.T @ VVC
  coef = x @ y # Faz a linearização com os dados
  eVVC = coef[0,0]*vm+coef[1,0] # VVC estimado
  el = abs(VVC - eVVC)
  elMax = np.max(el, 0)
  return (el)

def repetitividade(xi):
  return(np.std(xi,0,ddof=1).T) # Calcula o Desvio Padrão

def incertezaPadraoTipoA(st, n): # Incerteza Padrão do Tipo A
  return (np.asmatrix(st/mt.sqrt(n)))

def incertezaPadraoTipoB(uTB,caracteristicasPadrao): #Incerteza Padrão do Tipo B
  desc = uTB[0]
  y = uTB[1]
  tipo = re.search(r"^([^ |\(])+",desc)
  if tipo!=None:
    desc = tipo.group(0)
  if desc == 'Res':
    resp = y/(2*mt.sqrt(3))
  elif tipo[0] == 'Deriva':
    resp = y/mt.sqrt(3)
  elif tipo[0] == 'Lin':
    resp = y/mt.sqrt(3)
    #resp = zeros(size(y,1),1)
  elif tipo[0] == 'Hist':
    resp = y/(2*mt.sqrt(3))
  else: # "Quando for Incerteza - U"
    try:
      resp = y/caracteristicasPadrao['k'] # fatorAbrang
    except:
      resp = y
  return (resp)

def incertezaCombinada(y): # Cálculo da Incerteza Combinada
  return(np.sqrt(np.sum(np.square(y), axis=1)))

def grauLiberdadeEfetivo(uc,uTA,n,uTB,caracteristicasPadrao): # CÁLCULO DO GRAU DE LIBERDADE EFETIVO
  numerador = np.power(uc,4)
  veff = mb.repmat((np.asmatrix(caracteristicasPadrao['veff'])).T,1,np.size(uTB,1))
  va = mb.repmat(np.asmatrix([n-1]),np.size(uTA,0),1) # Grau de Liberdade
  denominador = (np.power(uTA,4)/va)+np.sum(np.power(uTB,4)/veff, axis=1)
  vefF = numerador/denominador # GRAU DE LIBERDADE EFETIVO
  return(vefF)

def fatorAbrangencia(nivelConfianca,veff):
  k=t.ppf(1-((1-nivelConfianca/100)/2),veff)
  return(k)

def incertezaExpandida(uc,k):#Cálculo da Incerteza Expandida
  ue = np.multiply(uc,k);
  return (ue)

def caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, xi_avanco, xi_atraso):
  em = erroMedicao(VVC, xi)
  es = erroSistematico(VVC, xi)
  ea = erroAleatorio(xi)
  rep = repetitividade(xi)
  lin = linearidade(VVC, xi)
  hist = histerese(xi_avanco,xi_atraso)

  uTA_padrao = incertezaPadraoTipoA(rep,np.size(xi,0))
  a = (np.asmatrix(np.array([incertezaPadraoTipoB(x,caracteristicasPadrao) for x in uTB]))).T
  b = incertezaPadraoTipoB(('Lin',lin),caracteristicasPadrao)
  c = incertezaPadraoTipoB(('Hist',hist),caracteristicasPadrao)
  uTB_padrao = np.concatenate((a, b, c), 1)
  uc = incertezaCombinada(np.concatenate((uTA_padrao, uTB_padrao),1))
  veff = grauLiberdadeEfetivo(uc,uTA_padrao,np.size(xi,0),uTB_padrao,caracteristicasPadrao)
  k = fatorAbrangencia((np.asmatrix(nivelConfiancaDesejado)).T,veff)
  ue = incertezaExpandida(uc,k)

  desc = ["EM","ES","EA","Rep","Hist","ELin","uTA (Rep_p)"]+[x[0]+"_p" for x in uTB]+["Lin_p",'Hist_p',"uC","Veff","k","uE"]
  data = np.concatenate((em, es, ea, rep, hist, lin, uTA_padrao, uTB_padrao, uc, veff, k, ue), 1)

  return(pd.DataFrame(data=data, columns=desc))

# Voltímetro

In [ ]:
caracteristicasPadrao = {'k': np.matrix([2, 2, 2, 2]),
                         'veff': np.matrix([np.Inf, np.Inf, np.Inf, np.Inf])}

uTB = [('Res',np.matrix([0.1, 0.1, 0.1, 0.1])),
       ('Res (p)',np.matrix([0.01, 0.01, 0.01, 0.01])),
       ('u (p)',np.matrix([0.001, 0.001, 0.001, 0.001])),
       ('Deriva (p)',np.matrix([0.002, 0.002, 0.002, 0.002]))]

nivelConfiancaDesejado = np.matrix([95.45,95.45,95.45,95.45])

VVC = np.matrix([25,50,75,100])

xi = np.matrix([[25.09, 50.07, 75.11, 100.15],
               [25.11, 50.05, 75.12, 100.14],
               [25.08, 50.06, 75.1, 100.17],
               [25.07, 50.04, 75.11, 100.16],
               [25.09, 50.06, 75.13, 100.14]])

caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, np.ones((1,np.size(xi,1))), np.ones((1,np.size(xi,1))))

,EM,ES,EA,Rep,Hist,ELin,uTA (Rep_p),Res_p,Res (p)_p,u (p)_p,Deriva (p)_p,Lin_p,Hist_p,uC,Veff,k,uE
0,0.11,0.088,0.013266,0.014832,0.0,0.022999,0.006633,0.028868,0.002887,0.0005,0.001155,0.013278,0.0,0.032612,2337.137329,2.001073,0.065260
1,0.07,0.056,0.010198,0.011402,0.0,0.033959,0.005099,0.028868,0.002887,0.0005,0.001155,0.019606,0.0,0.035407,9299.604873,2.000271,0.070823
2,0.13,0.114,0.010198,0.011402,0.0,0.001006,0.005099,0.028868,0.002887,0.0005,0.001155,0.000581,0.0,0.029489,4474.452709,2.000561,0.058994
3,0.17,0.152,0.011662,0.013038,0.0,0.011966,0.005831,0.028868,0.002887,0.0005,0.001155,0.006909,0.0,0.030413,2960.506313,2.000847,0.060853


# Paquímetro

In [ ]:
caracteristicasPadrao = {'k': np.matrix([2, 2, 2]),
                         'veff': np.matrix([np.Inf, np.Inf, np.Inf])}

uTB = [('Res',np.matrix([0.01, 0.01, 0.01])),
       ('Res (p)',np.matrix([0.001, 0.001, 0.001])),
       ('u (p)',np.matrix([0.001, 0.001, 0.001]))]

nivelConfiancaDesejado = np.matrix([95.45,95.45,95.45])

VVC = np.matrix([0.1, 0.5, 1.0])

xi_avanco = np.matrix([[0.101, 0.503, 1.005],
                       [0.102, 0.504, 1.004],
                       [0.102, 0.502, 1.006],
                       [0.103, 0.502, 1.005],
                       [0.101, 0.503, 1.004]])

xi_atraso = np.matrix([[0.098, 0.502, 1.002],
                       [0.1, 0.501, 1.0],
                       [0.099, 0.5, 1.003],
                       [0.099, 0.501, 1.001],
                       [0.1, 0.502, 1.002]])

xi = np.concatenate((xi_avanco, xi_atraso), 0)

caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, xi_avanco, xi_atraso)

# Balança

In [ ]:
caracteristicasPadrao = {'k': np.matrix([2, 2, 2.52]),
                         'veff': np.matrix([np.Inf, np.Inf, 6])}

uTB = [('Res',np.matrix([0.001, 0.001, 0.001])),
       ('u (p)',np.matrix([0.00001, 0.00001, 0.00002])),
       ('Deriva (p)',np.matrix([0.00001, 0.00001, 0.00001]))]

nivelConfiancaDesejado = np.matrix([95.45,95.45,95.45])

VVC = np.matrix([1,50,100])

xi = np.matrix([[1, 50, 100],
                [1.001, 50.001, 100],
                [1, 50.001, 100],
                [1, 50, 100.001],
                [1.001, 50.001, 100]])

caractEstatico(caracteristicasPadrao, uTB, nivelConfiancaDesejado, VVC, xi, np.ones((1,np.size(xi,1))), np.ones((1,np.size(xi,1))))

,EM,ES,EA,Rep,Hist,ELin,uTA (Rep_p),Res_p,u (p)_p,Deriva (p)_p,Lin_p,Hist_p,uC,Veff,k,uE
0,0.001,0.0004,0.000548,0.000245,0.0,0.000101,0.000110,0.000289,0.000005,0.000006,0.000058,0.0,0.000314,270.984055,2.009270,0.000631
1,0.001,0.0006,0.000548,0.000245,0.0,0.000199,0.000110,0.000289,0.000005,0.000006,0.000115,0.0,0.000330,327.818293,2.007657,0.000662
2,0.001,0.0002,0.000447,0.000200,0.0,0.000099,0.000089,0.000289,0.000008,0.000006,0.000057,0.0,0.000308,7.627150,2.387451,0.000735
